In [169]:
# Connection parameters
host = 'localhost'
user= 'postgres'
password='0000'
database='preprocess'  # the name of the batabase we are connecting
port = "5432"
# connection_string=f'postgresql://{user}:{password}@{host}:5432/{database}'

In [170]:
# !pip install psycopg2
# ! pip install psycopg2-binary

In [171]:
# prepare
import sqlalchemy
import psycopg2
import pandas as pd
import sqlite3
import pandas as pd
from io import StringIO
import csv
from psycopg2 import extras
import numpy as np

In [172]:
# Create a connection to the database
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

In [173]:
# Create a cursor object
cur = conn.cursor()

In [134]:
# Part 1: Build tables:  6 component-->6 tables (finish)

In [91]:
# 1) input itinerary component
cur.execute("""
DROP TABLE itinerary_input CASCADE
""")

In [92]:
# 1) input itinerary component
cur.execute("""
CREATE TABLE "itinerary_input" (
  "id" SERIAL PRIMARY KEY,
  "Orig_s" text NOT NULL,   
  "Dest_s" text NOT NULL,
  "depDay" varchar(10) NOT NULL
)
""")

In [93]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [108]:
# 2) market share (pie graph) component 
# consistent with itinerary, map, and recommendation
cur.execute("""
DROP TABLE marketshare CASCADE
""")

In [109]:
cur.execute("""
CREATE TABLE "marketshare" (
  "id" SERIAL PRIMARY KEY,
  "itinerary_input_id" INTEGER NOT NULL,
  "market_share" FLOAT NOT NULL,
  FOREIGN KEY ("itinerary_input_id") REFERENCES "itinerary_input" ("id")
)
""")


In [110]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [111]:
# 3) recommendation table component
# consistent with market share, map, and itinerary
cur.execute("""
DROP TABLE recommendation CASCADE
""")

In [112]:
cur.execute("""
CREATE TABLE "recommendation" (
  "id" SERIAL PRIMARY KEY,
  "itinerary_input_id" INTEGER NOT NULL,
  "dep_hour" INTEGER NOT NULL,
  "dep_min" INTEGER NOT NULL,
  "arr_hour" INTEGER NOT NULL,
  "arr_min" INTEGER NOT NULL,
  "option" TEXT,
  "elaptime" FLOAT NOT NULL,
  FOREIGN KEY ("itinerary_input_id") REFERENCES "itinerary_input" ("id") 
)
""")

In [113]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [114]:
# 4) map component
# consistent with market share, itinerary and recommendation
cur.execute("""
DROP TABLE map CASCADE
""")

In [115]:
cur.execute("""
CREATE TABLE "map" (
  "id" SERIAL PRIMARY KEY,
  "itinerary_input_id" INTEGER NOT NULL,
  "Orig_s" text NOT NULL,
  "Dest_s" text NOT NULL,
  FOREIGN KEY ("itinerary_input_id") REFERENCES "itinerary_input" ("id")
)
""")

In [116]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [117]:
# 5) result component (highest market_share itinerary info)
cur.execute("""
DROP TABLE result CASCADE
""")

In [118]:
cur.execute("""
CREATE TABLE "result" (
  "id" SERIAL PRIMARY KEY,
  "marketshare_id" INTEGER NOT NULL,
  "TOT_pax" FLOAT NOT NULL,
  "accuracy" FLOAT,
  FOREIGN KEY ("marketshare_id") REFERENCES "marketshare" ("id")
)
""")

In [119]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [120]:
cur.execute("""
DROP TABLE other_info CASCADE
""")

In [121]:
# 6) other info component (highest market_share itinerary info)
cur.execute("""
CREATE TABLE "other_info" (
  "id" SERIAL PRIMARY KEY,
  "marketshare_id" INTEGER NOT NULL,
  "detour" FLOAT NOT NULL,
  "stops" INTEGER NOT NULL,
  "real_dist" FLOAT NOT NULL,
  FOREIGN KEY ("marketshare_id") REFERENCES "marketshare" ("id")
)
""")

In [122]:
# Commit the transaction for the CREATE TABLE command
conn.commit()

In [43]:
# Part 2: import preprocessing (Prepro_v2) dataset to tables (current)
# data: preprocessing data

In [94]:
# 1) input itinerary component
df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')

In [95]:
# create column Orig_s, Dest_s
cf = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/country_to_num.csv')

# Merging to get Orig_s
df = df.merge(cf, left_on='Orig', right_on='code', how='left')
df.rename(columns={'country': 'Orig_s'}, inplace=True)
df.drop('code', axis=1, inplace=True)

# Merging to get Dest_s
df = df.merge(cf, left_on='Dest', right_on='code', how='left')
df.rename(columns={'country': 'Dest_s'}, inplace=True)
df.drop('code', axis=1, inplace=True)

In [96]:
df.drop(df.columns[0], axis=1, inplace=True)

In [97]:
df.drop(df.columns[-2], axis=1, inplace=True)
df.drop(df.columns[-3], axis=1, inplace=True)

In [98]:
df.head()

,Orig,con1,con2,Dest,op_flight1,op_flight2,op_flight3,depDay,elaptime,detour,...,cluster,TOT_pax,market_share,real_dist,total_time,connection_time,dep_hour,arr_hour,Orig_s,Dest_s
0,26,64,239,181,732.0,2624,1101,2.0,535.0,1.02676,...,111.0,0.0,0.0,3899.12,535.0,0.0,13,22,DE,SA
1,26,64,239,181,732.0,2624,1101,4.0,535.0,1.02676,...,111.0,0.0,0.0,3899.12,535.0,0.0,13,22,DE,SA
2,26,64,239,181,732.0,1748,1101,4.0,775.0,1.02676,...,111.0,0.0,0.0,3899.12,775.0,0.0,13,2,DE,SA
3,26,64,239,181,732.0,2624,1101,5.0,535.0,1.02676,...,111.0,0.0,0.0,3899.12,535.0,0.0,13,22,DE,SA
4,26,64,239,181,732.0,2624,1101,6.0,535.0,1.02676,...,111.0,0.0,0.0,3899.12,535.0,0.0,13,22,DE,SA


In [99]:
# deal with missing value


In [100]:
# Select only the relevant columns
df_selected = df[['Orig_s', 'Dest_s', 'depDay']]
# Convert 'depDay' from float to string if it's not in the datetime format
df_selected['depDay'] = df_selected['depDay'].astype(str)

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\3413273447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['depDay'] = df_selected['depDay'].astype(str)


In [101]:
# Convert the DataFrame to an in-memory buffer for copying to SQL
buffer = StringIO()
df_selected.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [102]:
# Use copy_from to copy the data to the database
cur.copy_from(buffer, 'itinerary_input', sep=',', columns=('Orig_s', 'Dest_s', 'depDay'))
conn.commit()

In [123]:
# 2) market share (pie graph) component
# 1) input itinerary component
# df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')
df_selected = df[['market_share']].astype(float)

In [124]:
# Retrieve all ids from the itinerary_input table
cur.execute("SELECT id FROM itinerary_input ORDER BY id")
itinerary_input_ids = cur.fetchall()

In [125]:
# Check if the number of rows matches
if len(df_selected) <= len(itinerary_input_ids):
    # Assign the itinerary_input_id values to the market_share DataFrame
    df_selected['itinerary_input_id'] = [id[0] for id in itinerary_input_ids[:len(df_selected)]]
else:
    raise ValueError("There are more market_share entries than itinerary_input_ids available.")

In [126]:
# Prepare the data for insertion
# Assuming the market_share_data DataFrame now has two columns: 'market_share' and 'itinerary_input_id'
# Convert the DataFrame to a list of tuples
data_tuples = list(df_selected.itertuples(index=False, name=None))

In [127]:
# Insert the data into the marketshare table
insert_query = 'INSERT INTO marketshare (market_share, itinerary_input_id) VALUES %s'
extras.execute_values(cur, insert_query, data_tuples)

In [128]:
conn.commit()

In [135]:
# 3) recommendation table component
# 1) input itinerary component
df_2 = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v3.csv')
# Select only the relevant columns
# Merge df with df_2 based on their index
df = df.join(df_2[['dep_min', 'arr_min']])

In [136]:
df.head()

,Orig,con1,con2,Dest,op_flight1,op_flight2,op_flight3,depDay,elaptime,detour,...,market_share,real_dist,total_time,connection_time,dep_hour,arr_hour,Orig_s,Dest_s,dep_min,arr_min
0,26,64,239,181,732.0,2624,1101,2.0,535.0,1.02676,...,0.0,3899.12,535.0,0.0,13,22,DE,SA,45,40
1,26,64,239,181,732.0,2624,1101,4.0,535.0,1.02676,...,0.0,3899.12,535.0,0.0,13,22,DE,SA,45,40
2,26,64,239,181,732.0,1748,1101,4.0,775.0,1.02676,...,0.0,3899.12,775.0,0.0,13,2,DE,SA,45,40
3,26,64,239,181,732.0,2624,1101,5.0,535.0,1.02676,...,0.0,3899.12,535.0,0.0,13,22,DE,SA,45,40
4,26,64,239,181,732.0,2624,1101,6.0,535.0,1.02676,...,0.0,3899.12,535.0,0.0,13,22,DE,SA,45,40


In [137]:
df_selected = df[['dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'elaptime']]
# define the data type 
df_selected['dep_hour'] = df_selected['dep_hour'].astype(int)
df_selected['dep_min'] = df_selected['dep_min'].astype(int)
df_selected['arr_hour'] = df_selected['arr_hour'].astype(int)
df_selected['arr_min'] = df_selected['arr_min'].astype(int)
df_selected['elaptime'] = df_selected['elaptime'].astype(float)

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\3531353423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['dep_hour'] = df_selected['dep_hour'].astype(int)
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\3531353423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['dep_min'] = df_selected['dep_min'].astype(int)
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\3531353423.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [138]:
# Retrieve all ids from the itinerary_input table
cur.execute("SELECT id FROM itinerary_input ORDER BY id")
itinerary_input_ids = cur.fetchall()

In [139]:
# Check if the number of rows matches
if len(df_selected) <= len(itinerary_input_ids):
    # Assign the itinerary_input_id values to the df_selected DataFrame
    df_selected['itinerary_input_id'] = [id_tuple[0] for id_tuple in itinerary_input_ids[:len(df_selected)]]
    df_selected['option'] = None  # Assuming 'option' is a placeholder for actual options to be filled in later.

    # Reorder the DataFrame columns to match the table schema
    df_selected = df_selected[['itinerary_input_id', 'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'option', 'elaptime']]

    # Convert the DataFrame to an in-memory buffer for copying to SQL
    buffer = StringIO()
    df_selected.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    # Use copy_from to copy the data to the database
    cur.copy_from(buffer, 'recommendation', sep=',', columns=('itinerary_input_id', 'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'option', 'elaptime'))
    conn.commit()
else:
    raise ValueError("There are more elaptime entries than itinerary_input_ids available.")

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\244383808.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['itinerary_input_id'] = [id_tuple[0] for id_tuple in itinerary_input_ids[:len(df_selected)]]
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\244383808.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['option'] = None  # Assuming 'option' is a placeholder for actual options to be filled in later.


In [72]:
# Note: Reorder can ensure the mismatch

In [160]:
# 4) map component
# 1) input itinerary component
# df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')
# Select only the relevant columns
df_selected = df[['Orig_s', 'Dest_s']]

In [161]:
# Retrieve all ids from the itinerary_input table
cur.execute("SELECT id FROM itinerary_input ORDER BY id")
itinerary_input_ids = cur.fetchall()

In [175]:
# based on Orig_s to set foreign key (itinerary_input_id)
# Check if the number of rows matches
# Assign the itinerary_input_id values to the df_selected DataFrame
df_selected['itinerary_input_id'] = [id_tuple[0] for id_tuple in itinerary_input_ids[:len(df_selected)]]
# Replace None with an empty string or a suitable placeholder
df_selected.fillna('', inplace=True)

# Ensure correct data types
# For example, if itinerary_input_id should be an integer
df_selected['itinerary_input_id'] = df_selected['itinerary_input_id'].astype(int)

# Ensuring the correct column order for df_selected
df_selected = df_selected[['itinerary_input_id', 'Orig_s', 'Dest_s']]

# Print the first few rows of df_selected to verify the data and column order
print(df_selected.head())

# Convert the DataFrame to an in-memory buffer for copying to SQL
buffer = StringIO()
df_selected.to_csv(buffer, index=False, header=False)
buffer.seek(0)

# Print the first few lines of the buffer to check its content
print(buffer.getvalue()[:500])

# Use copy_from to copy the data to the database
try:
    cur.copy_from(buffer, 'map', sep=',', columns=('itinerary_input_id', 'Orig_s', 'Dest_s'))
    conn.commit()
except Exception as e:
    print(f"Error during copy: {e}")
    conn.rollback()


C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\4082827936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['itinerary_input_id'] = [id_tuple[0] for id_tuple in itinerary_input_ids[:len(df_selected)]]
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\4082827936.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.fillna('', inplace=True)
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\4082827936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

   itinerary_input_id Orig_s Dest_s
0                   1     DE     SA
1                   2     DE     SA
2                   3     DE     SA
3                   4     DE     SA
4                   5     DE     SA
1,DE,SA
2,DE,SA
3,DE,SA
4,DE,SA
5,DE,SA
6,DE,SA
7,DE,SA
8,DE,SA
9,DE,SA
10,DE,SA
11,DE,SA
12,DE,SA
13,DE,SA
14,DE,SA
15,DE,SA
16,DE,SA
17,DE,SA
18,DE,SA
19,DE,SA
20,DE,SA
21,DE,SA
22,DE,SA
23,DE,SA
24,DE,SA
25,DE,SA
26,DE,SA
27,DE,SA
28,DE,SA
29,DE,SA
30,DE,SA
31,DE,SA
32,DE,SA
33,DE,SA
34,DE,SA
35,DE,SA
36,DE,SA
37,DE,SA
38,DE,SA
39,DE,SA
40,DE,SA
41,DE,SA
42,DE,OM
43,DE,OM
44,DE,OM
45,DE,OM
46,DE,OM
47,DE,OM
48,DE,OM
49,DE,OM
50,DE,OM
51,DE,OM


In [176]:
# 5) result component (highest market_share itinerary info)
# df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')
# Select only the relevant columns
df_selected = df[['TOT_pax']]

In [177]:
# Retrieve all ids from the recommendation table
cur.execute("SELECT id FROM marketshare ORDER BY id")
marketshare_ids = cur.fetchall()

In [178]:
if len(df_selected) <= len(marketshare_ids):
    # Assign the itinerary_input_id values to the df_selected DataFrame
    df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]

    # Reorder the DataFrame columns to match the table schema
    df_selected = df_selected[['marketshare_id', 'TOT_pax']]
    # Make sure to assign np.nan instead of None to create proper NaN values for float columns
    df_selected['accuracy'] = np.nan

    # Convert the DataFrame to an in-memory buffer for copying to SQL
    buffer = StringIO()
    df_selected.to_csv(buffer, index=False, header=False, na_rep='\\N') 
    # Use na_rep to replace NaN with \N in the CSV
    buffer.seek(0)

    # Use copy_from to copy the data to the database
    cur.copy_from(buffer, 'result', sep=',', columns=('marketshare_id', 'TOT_pax', 'accuracy'))
    conn.commit()
else:
    raise ValueError("There are more entries than fk_constrain available.")

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\1200834003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]


In [179]:
# 6) other info component (highest market_share itinerary info)
# 1) input itinerary component
# df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')

In [180]:
# Create a function to determine the number of stops based on the columns
#def calculate_stops(row):
    # Check each 'stops_X.X' column starting from the highest
    #for i in range(2, -1, -1):
        #if row[f'stops_{i}.0'] == 1:
            #return i
    #return 0  # Return 0 if none of the stops columns have a 1

In [181]:
# Apply the function to each row to create the 'stops' column
# df['stops'] = df.apply(calculate_stops, axis=1)

In [182]:
# Select only the relevant columns
df_selected = df[['detour', 'stops', 'real_dist']]
# define the data type 
df_selected['detour'] = df_selected['detour'].astype(float)
df_selected['stops'] = df_selected['stops'].astype(int)
df_selected['real_dist'] = df_selected['real_dist'].astype(float)

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\78121940.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['detour'] = df_selected['detour'].astype(float)
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\78121940.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['stops'] = df_selected['stops'].astype(int)
C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\78121940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [183]:
# Retrieve all ids from the recommendation table
cur.execute("SELECT id FROM marketshare ORDER BY id")
marketshare_ids = cur.fetchall()

In [184]:
if len(df_selected) <= len(marketshare_ids):
    # Assign the itinerary_input_id values to the df_selected DataFrame
    df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]

    # Reorder the DataFrame columns to match the table schema
    df_selected = df_selected[['marketshare_id', 'detour', 'stops', 'real_dist']]

    # Convert the DataFrame to an in-memory buffer for copying to SQL
    buffer = StringIO()
    df_selected.to_csv(buffer, index=False, header=False) 
    buffer.seek(0)

    # Use copy_from to copy the data to the database
    cur.copy_from(buffer, 'other_info', sep=',', columns=('marketshare_id', 'detour', 'stops', 'real_dist'))
    conn.commit()
else:
    raise ValueError("There are more entries than fk_constrain available.")

C:\Users\Alla\AppData\Local\Temp\ipykernel_14396\2278559781.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]


In [185]:
# Close the cursor and connection
cur.close()
conn.close()